In [20]:
import numpy as np
import sys
import geopandas as gpd

sys.path.append("/home/nilscp/GIT/crater_morphometry")
import geomorphometry
from preprocessing import DEM_extraction
from rim_detection import rim_detection
from pathlib import Path

sys.path.append("/home/nilscp/GIT/")
from rastertools import utils

## Clip DEM and visible image for rayed craters (SLDEM2015)

In [6]:
lonlat_crs = ('GEOGCRS["Moon 2000",DATUM["D_Moon_2000",'
              'ELLIPSOID["Moon_2000_IAU_IAG",1737400,0,'
              'LENGTHUNIT["metre",1,ID["EPSG",9001]]]],'
              'PRIMEM["Greenwich",0,'
              'ANGLEUNIT["Decimal_Degree",0.0174532925199433]],'
              'CS[ellipsoidal,2],AXIS["longitude",east,ORDER[1],'
              'ANGLEUNIT["Decimal_Degree",0.0174532925199433]],'
              'AXIS["latitude",north,ORDER[2],'
              'ANGLEUNIT["Decimal_Degree",0.0174532925199433]]]')

In [3]:
location_of_craters ="/home/nilscp/QGIS/Moon/Stopar2017/fresh_impact_craters_RANGER_validation.shp"

Let's add lon and lat in degrees

In [4]:
gdf = gpd.read_file(location_of_craters)

In [7]:
gdf_lonlat = gdf.to_crs(lonlat_crs)

In [14]:
lon = []
lat = []
for i, row in gdf_lonlat.iterrows():
    lon.append(np.array(row.geometry.coords.xy[0])[0])
    lat.append(np.array(row.geometry.coords.xy[1])[0])

In [15]:
gdf["lon"] = lon
gdf["lat"] = lat

In [17]:
gdf.to_file("/home/nilscp/QGIS/Moon/Stopar2017/fresh_impact_craters_RANGER_validation_lonlat.shp")

Let's check the crs of the DEM is the same as the crs of the shapefile

In [19]:
gdf.crs.to_proj4()

/home/nilscp/.pyenv/versions/3.8.0/envs/computer-vision/lib/python3.8/site-packages/pyproj/crs/crs.py:1256: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  return self._crs.to_proj4(version=version)


'+proj=eqc +lat_ts=9 +lat_0=0 +lon_0=180 +x_0=0 +y_0=0 +R=1737400 +units=m +no_defs +type=crs'

In [23]:
location_of_craters = "/home/nilscp/QGIS/Moon/Stopar2017/fresh_impact_craters_RANGER_validation_lonlat.shp"
dem = "/home/nilscp/QGIS/Moon/NAC_DTM/NAC_DTM_RANGER.tiff"
orthoimage = "/home/nilscp/QGIS/Moon/NAC_DTM/NAC_DTM_RANGER_M144490730_50CM.tiff"
clip_distance = 4.0
output_dir = "/home/nilscp/tmp/NAC_DTM_RANGER/DTM_clipped/"
output_dir_ortho = "/home/nilscp/tmp/NAC_DTM_RANGER/Ortho_clipped/"
identifier_dem = "NACDTM"
identifier_orthoimage = "NAC"
shp_folder = "/home/nilscp/tmp/NAC_DTM_RANGER/shapefiles/"

In [22]:
utils.get_raster_crs(dem).to_proj4()

'+proj=eqc +lat_ts=9 +lat_0=0 +lon_0=180 +x_0=0 +y_0=0 +R=1737400 +units=m +no_defs=True'

In [24]:
DEM_extraction.clip_raster_to_crater(location_of_craters, dem, clip_distance, output_dir, shp_folder, identifier_dem, craterID = None)

100%|██████████| 22/22 [00:22<00:00,  1.04s/it]


In [25]:
DEM_extraction.clip_raster_to_crater(location_of_craters, orthoimage, clip_distance, output_dir_ortho, shp_folder, identifier_orthoimage, craterID = None)

100%|██████████| 22/22 [00:23<00:00,  1.08s/it]


## Generate regional (2.0-3.0R) detrended DEM (SLDEM2015, Rayed craters)

In [27]:
location_of_craters = "/home/nilscp/QGIS/Moon/Stopar2017/fresh_impact_craters_RANGER_validation_lonlat.shp"
scaling_factor = 1.0
dem_folder = "/home/nilscp/tmp/NAC_DTM_RANGER/DTM_clipped/"
dem_detrended_folder = "/home/nilscp/tmp/NAC_DTM_RANGER/DTM_clipped_detrended/"
craterID = None
overwrite = True
shp_folder = "/home/nilscp/tmp/NAC_DTM_RANGER/shapefiles/"

In [28]:
rim_detection.generated_detrended_dem(location_of_craters, scaling_factor, dem_folder, shp_folder, dem_detrended_folder, craterID=craterID, overwrite=overwrite)

  0%|          | 0/22 [00:00<?, ?it/s]/home/nilscp/.pyenv/versions/3.8.0/envs/computer-vision/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/nilscp/.pyenv/versions/3.8.0/envs/computer-vision/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1664: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
100%|██████████| 22/22 [00:08<00:00,  2.58it/s]


## The detection of the rim composite, and the following update of the crater centre
----
#### Note that the algorithm will not work if:
- the centre of the crater is outside of the crater of interest
- the resolution of the dem is too low (at least 20 cells is required along the crater radius to provide reasonable results <br>
----
#### It can be that the algorithm work but gives wrong results if:
- There are large change in slopes that does not correspond to the actual change in slope associated with the rim (either very close or very far)

In [31]:
location_of_craters = "/home/nilscp/QGIS/Moon/Stopar2017/fresh_impact_craters_RANGER_validation_lonlat.shp"
dem_folder = "/home/nilscp/tmp/NAC_DTM_RANGER/DTM_clipped_detrended/"
shp_folder = "/home/nilscp/tmp/NAC_DTM_RANGER/shapefiles/"
suffix = "_NACDTM_detrended.tif"

In [32]:
rim_detection.main(location_of_craters, dem_folder, shp_folder, suffix, threshold_min=None, threshold_max=None , craterID = None)

  0%|          | 0/22 [00:00<?, ?it/s]/home/nilscp/GIT/crater_morphometry/rim_detection/rim_detection.py:1033: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(filename)
100%|██████████| 22/22 [01:42<00:00,  4.65s/it]


## Manual check of results
It is important to manually check if the detected rims give meaningful results. If not, the polygon can be deleted, and replace by a manually generated ellipse. This should (hopefully) not happen in many cases for fresh impact structures (<5%). However, this is likely to happen more often for degraded craters (such as the rayed crater dataset, that contains both young and old craters). For completion, we are writing down the id of the crater where a problem is detected, to understand if we can improve the rule-based detection of rim in the future. 

## Update centre of craters
- Merge the centres of craters together



In [3]:
out_shapefile = "/home/nilscp/tmp/NAC_DTM_RANGER/shapefiles/final_crater_center/global/fresh_impact_craters_RANGER_validation_lonlat_manually_checked.shp"
shp_folder = "/home/nilscp/tmp/NAC_DTM_RANGER/shapefiles/"
old_crater_centres = "/home/nilscp/QGIS/Moon/Stopar2017/fresh_impact_craters_RANGER_validation_lonlat.shp"

In [ ]:
rim_detection.update_crater_centres(shp_folder, out_shapefile, old_crater_centres, replace_crs=False)

## Update the DEM, and the 2.0R-3.0R and 0.9R-1.1R detrendings
- decrease clip distance to 4R to divide the size of DEMs by 2 (for SLDEM2015 and SLDEM2013)
- could be that the size of the SLDEM2013 could be decreased by moving away from float.

In [33]:
location_of_craters = "/home/nilscp/tmp/NAC_DTM_RANGER/shapefiles/final_crater_center/global/fresh_impact_craters_RANGER_validation_lonlat_manually_checked.shp"
dem = "/home/nilscp/QGIS/Moon/NAC_DTM/NAC_DTM_RANGER.tiff"
orthoimage = "/home/nilscp/QGIS/Moon/NAC_DTM/NAC_DTM_RANGER_M144490730_50CM.tiff"
clip_distance = 4.0
output_dir = "/home/nilscp/tmp/NAC_DTM_RANGER/DTM_clipped/"
output_dir_ortho = "/home/nilscp/tmp/NAC_DTM_RANGER/Ortho_clipped/"
identifier_dem = "NACDTM"
identifier_orthoimage = "NAC"
shp_folder = "/home/nilscp/tmp/NAC_DTM_RANGER/shapefiles/final_crater_center/"

In [34]:
DEM_extraction.clip_raster_to_crater(location_of_craters, dem, clip_distance, output_dir, shp_folder, identifier_dem, craterID = None)

100%|██████████| 22/22 [00:25<00:00,  1.17s/it]


In [35]:
DEM_extraction.clip_raster_to_crater(location_of_craters, orthoimage, clip_distance, output_dir_ortho, shp_folder, identifier_orthoimage, craterID = None)

100%|██████████| 22/22 [00:28<00:00,  1.28s/it]


## Update the 2.0R-3.0R and 0.9R-1.1R detrendings

In [36]:
scaling_factor = 1.0
dem_folder = "/home/nilscp/tmp/NAC_DTM_RANGER/DTM_clipped/"
dem_detrended_folder = "/home/nilscp/tmp/NAC_DTM_RANGER/DTM_clipped_detrended/"
craterID = None
overwrite = True
shp_folder = "/home/nilscp/tmp/NAC_DTM_RANGER/shapefiles/final_crater_center/"

In [37]:
rim_detection.generated_detrended_dem(location_of_craters, scaling_factor, dem_folder, shp_folder, dem_detrended_folder, craterID=craterID, overwrite=overwrite)

100%|██████████| 22/22 [00:09<00:00,  2.33it/s]


## Calculate morphometrical parameters from the second ellipse, final crater centre and new detrended products
- load ellipse in world coordinates (can also give the possibility to use points and not the ellipse!)
- create an ellipse with 2.0 r (maybe need to modify with the -xc, yc

There are a few problems by using ellipse... approximating the rim of a crater with an ellipse is totally ok, but it will always result in "shooting" over or under the actual position of the rim of crater. The majority of morphometric parameters (e.g., middle, upper, rim flank slope) are a bit sensitive to the exact location of the rim. I think I will try to create a function that retrieve the breaks in slope within 10% of the ellipse, and then use those few locations to extract cross sections and calculate parameters. This means that not all of the craters will have a constant number of cross-sections (512) as for ellipses. I will compare the results of those different values.

In [ ]:
location_of_craters = "/home/nilscp/tmp/NAC_DTM_RANGER/shapefiles/final_crater_center/global/fresh_impact_craters_RANGER_validation_lonlat_manually_checked_noproj.shp"
shp_folder = "/home/nilscp/tmp/NAC_DTM_RANGER/shapefiles/"
dem_folder = "/home/nilscp/tmp/NAC_DTM_RANGER/DTM_clipped_detrended/"
suffix = "_NACDTM_detrended.tif"
prefix = 'crater'
craterID=None